# TWAS
a) Manhattan plot of glyco-TWAS results (unadjusted data); annotate points to indicate MR significance



## Combine Data
TWAS and MR results generaated by pipelines were combined using the following commands
```
python /mnt/lustre/home/rl3328/rl3328/motor_qtl/combine_twas.py /mnt/lustre/lab/ctcn/hklein/motor_qtl_project/unadjusted_wp_2020/susie_twas/noQC_nonimputed_result/twas --type both --output ROSMAP_eQTL_wp_unadjusted
```
### Input
Input files are all on the new AWS-based HPC
* Combined TWAS file:`/mnt/lustre/lab/ctcn/hklein/motor_qtl_project/wp_2020/susie_twas/noQC_nonimputed_result/twas/ROSMAP_eQTL_wp_unadjusted.combined_twas.tsv.gz`
* Combined MR file:`/mnt/lustre/lab/ctcn/hklein/motor_qtl_project/unadjusted_wp_2020/susie_twas/noQC_nonimputed_result/twas/ROSMAP_eQTL_wp_unadjusted.combined_mr.tsv.gz`
* reference file for gene annotation: `/mnt/lustre/home/rl3328/rl3328/resource/Homo_sapiens.GRCh38.103.chr.reformatted.collapse_only.gene.region_list.2025`


### Landscape

* The TWAS results show substantial variation in the number of available genes across contexts. Monocyte, DLPFC, and AC have notably fewer genes available for analysis. In particular, the DLPFC bulk eQTL dataset includes only 57 genes, which likely explains why no genes reach significance under the most accurate predictive model for DLPFC bulk eQTL.
* Consistently, the MR analysis also yields no significant genes for monocyte, DLPFC, or AC.

* **Given these limitations, it may be more appropriate to combine bulk and single-nucleus (sn) data for DLPFC in downstream analyses.**



#### TWAS

In [6]:
library(data.table)
library(tidyverse)
twas <- fread('/mnt/lustre/lab/ctcn/hklein/motor_qtl_project/unadjusted_wp_2020/susie_twas/noQC_nonimputed_result/twas/ROSMAP_eQTL_wp_unadjusted.combined_twas.tsv.gz')

In [7]:
head(twas)
dim(twas)

chr,molecular_id,TSS,start,end,context,gwas_study,method,is_imputable,is_selected_method,rsq_cv,pval_cv,twas_z,twas_pval,type,block,method_selected_original,region,study_context,source_file
<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>
10,ENSG00000055950,100987515,99320000,102120000,DLPFC_DeJager_eQTL,unadjusted_wp_2020,bayes_l,TRUE,FALSE,0.1507800,1.287865e-29,-0.5663673,0.5711441,eQTL,chr10_100331627_104378781,FALSE,chr10_100331627_104378781,ROSMAP_eQTL_wp,ROSMAP_eQTL_wp_unadjusted.chr10_100331627_104378781.twas.tsv.gz
10,ENSG00000055950,100987515,99320000,102120000,DLPFC_DeJager_eQTL,unadjusted_wp_2020,bayes_r,TRUE,FALSE,0.1523213,6.298261e-30,-0.5729437,0.5666828,eQTL,chr10_100331627_104378781,FALSE,chr10_100331627_104378781,ROSMAP_eQTL_wp,ROSMAP_eQTL_wp_unadjusted.chr10_100331627_104378781.twas.tsv.gz
10,ENSG00000055950,100987515,99320000,102120000,DLPFC_DeJager_eQTL,unadjusted_wp_2020,enet,TRUE,TRUE,0.1586583,3.283208e-31,-0.1316813,0.8952364,eQTL,chr10_100331627_104378781,TRUE,chr10_100331627_104378781,ROSMAP_eQTL_wp,ROSMAP_eQTL_wp_unadjusted.chr10_100331627_104378781.twas.tsv.gz
10,ENSG00000055950,100987515,99320000,102120000,DLPFC_DeJager_eQTL,unadjusted_wp_2020,lasso,TRUE,FALSE,0.1470135,7.357674e-29,-0.1149415,0.9084915,eQTL,chr10_100331627_104378781,FALSE,chr10_100331627_104378781,ROSMAP_eQTL_wp,ROSMAP_eQTL_wp_unadjusted.chr10_100331627_104378781.twas.tsv.gz
10,ENSG00000055950,100987515,99320000,102120000,DLPFC_DeJager_eQTL,unadjusted_wp_2020,mrash,TRUE,FALSE,0.1283372,3.746242e-25,-0.4621670,0.6439616,eQTL,chr10_100331627_104378781,FALSE,chr10_100331627_104378781,ROSMAP_eQTL_wp,ROSMAP_eQTL_wp_unadjusted.chr10_100331627_104378781.twas.tsv.gz
10,ENSG00000055950,100987515,99320000,102120000,DLPFC_DeJager_eQTL,unadjusted_wp_2020,susie,TRUE,FALSE,0.1577097,5.116002e-31,-0.1690379,0.8657668,eQTL,chr10_100331627_104378781,FALSE,chr10_100331627_104378781,ROSMAP_eQTL_wp,ROSMAP_eQTL_wp_unadjusted.chr10_100331627_104378781.twas.tsv.gz


[1] 44820    20

In [8]:
## TWAS analysis were performed across 10 contexts and 13872 genes
twas %>% pull(context) |> unique()#%>%length

twas %>% pull(molecular_id)%>%unique()%>%length



[1] "DLPFC_DeJager_eQTL"

[1] 7443

In [9]:
## available genes by contexts
twas |> group_by(context) |> summarize(gene = n_distinct(molecular_id))

context,gene
<chr>,<int>
DLPFC_DeJager_eQTL,7443


In [10]:

twas_res_sig <- twas %>% filter(is_selected_method=="TRUE")%>%filter(twas_pval<=0.05)#%>%pull(context)%>%unique()
twas_res_sig|> pull(molecular_id)%>%unique()%>%length


[1] 1213

In [11]:
twas_DLPFC_bulk = twas |> filter(context == 'DLPFC_DeJager_eQTL') |> pull(molecular_id)%>%unique()%>%length
twas_DLPFC_bulk

[1] 7443

#### MR

In [13]:
mr <- fread('/mnt/lustre/lab/ctcn/hklein/motor_qtl_project/unadjusted_wp_2020/susie_twas/noQC_nonimputed_result/twas/ROSMAP_eQTL_wp_unadjusted.combined_mr.tsv.gz')

In [14]:
mr_sig_unadj <- mr %>% .[
  meta_pval < 0.05 / .N &          # Bonferroni correction
  cpip >= 0.7 &                    # strong causal evidence
  num_CS >= 1 &                    # as quested by Hans and Gao on Oct 24, 2025 to get a loose version
  Q_pval > 0.01 &                  # no heterogeneity
  I2 < 0.4                         # low heterogeneity
]

In [15]:
head(mr_sig_unadj)
dim(mr_sig_unadj)

Q,Q_pval,I2,context,cpip,gene_name,gwas_study,meta_eff,meta_pval,num_CS,num_IV,se_meta_eff,region,study_context,source_file
<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<int>,<int>,<dbl>,<chr>,<chr>,<chr>
0.134,0.715,0,DLPFC_DeJager_eQTL,0.960,ENSG00000171206,unadjusted_wp_2020,0.001,0,2,71,0,chr10_100331627_104378781,ROSMAP_eQTL_wp,ROSMAP_eQTL_wp_unadjusted.chr10_100331627_104378781.mr_result.tsv.gz
0.530,0.467,0,DLPFC_DeJager_eQTL,0.989,ENSG00000203780,unadjusted_wp_2020,0.000,0,2,19,0,chr10_125506866_127341896,ROSMAP_eQTL_wp,ROSMAP_eQTL_wp_unadjusted.chr10_125506866_127341896.mr_result.tsv.gz
0.000,1.000,0,DLPFC_DeJager_eQTL,1.024,ENSG00000148572,unadjusted_wp_2020,-0.001,0,1,89,0,chr10_62446953_64035328,ROSMAP_eQTL_wp,ROSMAP_eQTL_wp_unadjusted.chr10_62446953_64035328.mr_result.tsv.gz
0.000,1.000,0,DLPFC_DeJager_eQTL,0.950,ENSG00000165644,unadjusted_wp_2020,0.002,0,1,18,0,chr10_74769575_77264695,ROSMAP_eQTL_wp,ROSMAP_eQTL_wp_unadjusted.chr10_74769575_77264695.mr_result.tsv.gz
0.313,0.855,0,DLPFC_DeJager_eQTL,0.954,ENSG00000188199,unadjusted_wp_2020,0.000,0,3,9,0,chr10_79309511_80126158,ROSMAP_eQTL_wp,ROSMAP_eQTL_wp_unadjusted.chr10_79309511_80126158.mr_result.tsv.gz
1.383,0.847,0,DLPFC_DeJager_eQTL,0.965,ENSG00000228570,unadjusted_wp_2020,0.000,0,5,21,0,chr10_79309511_80126158,ROSMAP_eQTL_wp,ROSMAP_eQTL_wp_unadjusted.chr10_79309511_80126158.mr_result.tsv.gz


[1] 80 15

In [16]:
mr_sig_unadj |> pull(gene_name)%>%unique()%>%length

[1] 80

In [17]:
mr_sig_unadj |> group_by(context) |> summarize(gene = n_distinct(gene_name))

context,gene
<chr>,<int>
DLPFC_DeJager_eQTL,80


In [37]:
## Regardless of context, there are 60 genes have both TWAS significant and MR significant asscoaitions
intersect(twas_unadj_sig$molecular_id, mr_sig_unadj$gene_name) |> unique()|> length()

[1] 36

### summarize significant TWAS & MR genes

* Significant TWAS associations were defined using two criteria: (i) the most accurate prediction model yielded a TWAS p-value below the Bonferroni-corrected threshold, or (ii) at least half of the prediction models produced p-values below this threshold.
* Among the 7,443 total genes from DLPFC bulk eQTL, 67 genes show significant TWAS associations

* Mendelian randomization (MR) associations were considered significant if they met all of the following criteria: a meta-analysis p-value passing Bonferroni correction (meta-analysis p < 0.05 divided by the number of tested genes), strong causal evidence as indicated by a causal posterior inclusion probability (CPIP) ≥ 0.7, at least one credible set (num_CS ≥ 1), and no evidence of substantial heterogeneity (Cochran’s Q test p > 0.01 and I² < 0.4).
* For DLPFC bulk eQTL, 80 genes show significant MR associations.
* For DLPFC bulk eQTL, 36 genes are both MR and TWAS significant. 
* KAT8, SLC39A13 and ZNF668 are significant TWAS gene for both gait speed and AD

In [19]:
n_imputable_genes <- twas %>%
  filter(is_imputable) %>%                  # keep only imputable rows
  distinct(molecular_id, context, gwas_study) %>%         # count unique gene-context pairs
  nrow()
n_imputable_genes

[1] 7443

In [20]:
# Compute cutoff using Bonferroni-style correction
unadj_cutoff <- 0.05 / n_imputable_genes
unadj_cutoff

[1] 6.717721e-06

In [21]:
twas_unadj_sig <- twas %>%
  mutate(sig = twas_pval < unadj_cutoff) %>%
  group_by(molecular_id, context, gwas_study) %>%
  summarise(
    n_methods = n(),
    n_sig = sum(sig, na.rm = T),
    best_sig = any(sig & is_selected_method, na.rm = T),
    .groups = "drop"
  ) %>%
  mutate(pass = (n_sig / n_methods >= 0.5) | best_sig) %>%
  mutate(pass_strict = (n_sig / n_methods >= 0.5)) %>%
  filter(pass) %>% left_join(twas) %>% arrange(twas_pval) %>% distinct(molecular_id, .keep_all = T)
twas_unadj_sig %>% dim

Joining with `by = join_by(molecular_id, context, gwas_study)`


[1] 67 25

In [22]:
gene_ref <- fread('/mnt/lustre/home/rl3328/rl3328/resource/Homo_sapiens.GRCh38.103.chr.reformatted.collapse_only.gene.region_list.2025')

In [24]:
# twas_unadj_sig |> left_join(gene_ref, by = c("molecular_id" = "gene_id")) |> select(molecular_id, gene_name, everything())


## Figure data

In [25]:
df <- twas %>%
  mutate(sig = twas_pval < unadj_cutoff) %>%
  group_by(molecular_id, context, gwas_study) %>%
  summarise(
    n_methods = n(),
    n_sig = sum(sig, na.rm = TRUE),
    best_pval = min(twas_pval, na.rm = TRUE),
    best_method_pval = min(twas_pval[is_selected_method], na.rm = TRUE),
    best_method_selected = any(sig & is_selected_method, na.rm = TRUE),
    .groups = "drop"
  ) %>%
  mutate(
    pass = (n_sig / n_methods >= 0.5) | best_method_selected, # pass defined by two threshold, best or half methods satisfied
    final_pval = ifelse(n_sig / n_methods >= 0.5, best_pval, best_method_pval),
    # fallback: if no selected method existed (Inf), use best_pval
    final_pval = ifelse(is.infinite(final_pval), best_pval, final_pval)
  ) %>%
  left_join(twas, by = c("molecular_id", "context", "gwas_study")) %>%
  filter(twas_pval == final_pval) %>%
  distinct(molecular_id, .keep_all = TRUE) %>%
  mutate(
    logp = -log10(twas_pval) * sign(twas_z),
    pos = (start + end) / 2
  ) %>%
  merge(gene_ref, by.x = 'molecular_id', by.y = 'gene_id')

# Order chromosomes
df$chr <- factor(df$chr, levels = as.character(1:22))

Warning message:
“There were 34 warnings in `summarise()`.
The first warning was:
ℹ In argument: `best_method_pval = min(twas_pval[is_selected_method], na.rm =
  TRUE)`.
ℹ In group 113: `molecular_id = "ENSG00000011465"`, `context =
  "DLPFC_DeJager_eQTL"`, `gwas_study = "unadjusted_wp_2020"`.
Caused by warning in `min()`:
! no non-missing arguments to min; returning Inf
ℹ Run `dplyr::last_dplyr_warnings()` to see the 33 remaining warnings.”


In [26]:
dim(df)

[1] 7443   33

In [27]:
head(df)

,molecular_id,context,gwas_study,n_methods,n_sig,best_pval,best_method_pval,best_method_selected,pass,final_pval,⋯,method_selected_original,region,study_context,source_file,logp,pos,#chr,TSS.y,TES,gene_name
,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<lgl>,<lgl>,<dbl>,⋯,<lgl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<int>,<int>,<chr>
1,ENSG00000000457,DLPFC_DeJager_eQTL,unadjusted_wp_2020,6,0,0.506484945,0.506484945,FALSE,FALSE,0.506484945,⋯,TRUE,chr1_168438717_170228106,ROSMAP_eQTL_wp,ROSMAP_eQTL_wp_unadjusted.chr1_168438717_170228106.twas.tsv.gz,0.2954335,169540000,chr1,169894267,169849631,SCYL3
2,ENSG00000000460,DLPFC_DeJager_eQTL,unadjusted_wp_2020,6,0,0.400370493,0.633623327,FALSE,FALSE,0.633623327,⋯,TRUE,chr1_168438717_170228106,ROSMAP_eQTL_wp,ROSMAP_eQTL_wp_unadjusted.chr1_168438717_170228106.twas.tsv.gz,0.1981688,169540000,chr1,169662007,169854080,C1orf112
3,ENSG00000000971,DLPFC_DeJager_eQTL,unadjusted_wp_2020,6,0,0.006181974,0.007051131,FALSE,FALSE,0.007051131,⋯,TRUE,chr1_195599253_199271134,ROSMAP_eQTL_wp,ROSMAP_eQTL_wp_unadjusted.chr1_195599253_199271134.twas.tsv.gz,2.1517412,195093752,chr1,196652043,196747504,CFH
4,ENSG00000001084,DLPFC_DeJager_eQTL,unadjusted_wp_2020,6,0,0.147556313,0.502194806,FALSE,FALSE,0.502194806,⋯,TRUE,chr6_52730905_54027603,ROSMAP_eQTL_wp,ROSMAP_eQTL_wp_unadjusted.chr6_52730905_54027603.twas.tsv.gz,0.2991278,53557156,chr6,53616970,53497341,GCLC
5,ENSG00000001460,DLPFC_DeJager_eQTL,unadjusted_wp_2020,6,0,0.659517185,0.659517185,FALSE,FALSE,0.659517185,⋯,TRUE,chr1_24199848_26390306,ROSMAP_eQTL_wp,ROSMAP_eQTL_wp_unadjusted.chr1_24199848_26390306.twas.tsv.gz,-0.1807739,24386966,chr1,24416934,24356999,STPG1
6,ENSG00000001461,DLPFC_DeJager_eQTL,unadjusted_wp_2020,6,0,0.414763094,0.778955410,FALSE,FALSE,0.778955410,⋯,TRUE,chr1_24199848_26390306,ROSMAP_eQTL_wp,ROSMAP_eQTL_wp_unadjusted.chr1_24199848_26390306.twas.tsv.gz,-0.1084874,24444388,chr1,24415802,24472976,NIPAL3


In [28]:
chr_info <- df %>%
  group_by(chr) %>%
  summarise(chr_len = max(pos)) %>%
  mutate(chr_start = lag(cumsum(chr_len), default = 0))

df <- df %>%
  left_join(chr_info, by = "chr") %>%
  mutate(cum_pos = pos + chr_start)
  
axis_df <- chr_info %>%
  mutate(center = chr_start + chr_len/2)


In [29]:
df <- df %>%
  left_join(
    mr_sig_unadj %>% rename(molecular_id = gene_name),
    by = c("molecular_id", "context")
  )

In [31]:
# df %>% filter(!is.na(Q_pval)) %>% select(molecular_id, context, gene_name)

In [32]:
head(df)
dim(df)

,molecular_id,context,gwas_study.x,n_methods,n_sig,best_pval,best_method_pval,best_method_selected,pass,final_pval,⋯,cpip,gwas_study.y,meta_eff,meta_pval,num_CS,num_IV,se_meta_eff,region.y,study_context.y,source_file.y
,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<lgl>,<lgl>,<dbl>,⋯,<dbl>,<chr>,<dbl>,<dbl>,<int>,<int>,<dbl>,<chr>,<chr>,<chr>
1,ENSG00000000457,DLPFC_DeJager_eQTL,unadjusted_wp_2020,6,0,0.506484945,0.506484945,FALSE,FALSE,0.506484945,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,ENSG00000000460,DLPFC_DeJager_eQTL,unadjusted_wp_2020,6,0,0.400370493,0.633623327,FALSE,FALSE,0.633623327,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,ENSG00000000971,DLPFC_DeJager_eQTL,unadjusted_wp_2020,6,0,0.006181974,0.007051131,FALSE,FALSE,0.007051131,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,ENSG00000001084,DLPFC_DeJager_eQTL,unadjusted_wp_2020,6,0,0.147556313,0.502194806,FALSE,FALSE,0.502194806,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,ENSG00000001460,DLPFC_DeJager_eQTL,unadjusted_wp_2020,6,0,0.659517185,0.659517185,FALSE,FALSE,0.659517185,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,ENSG00000001461,DLPFC_DeJager_eQTL,unadjusted_wp_2020,6,0,0.414763094,0.778955410,FALSE,FALSE,0.778955410,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


[1] 7443   49

In [33]:

# Add a column for MR significance (example: FDR < 0.05)
df <- df %>%
  mutate(category = case_when(
    !is.na(Q_pval) & pass ~ "TWAS & MR",        # Both significant
    is.na(Q_pval) & pass ~ "TWAS only",         # Only TWAS significant
    !is.na(Q_pval) & !pass ~ "MR only",         # Only MR significant
    is.na(Q_pval) & !pass ~ "Not Significant",  # Neither significant
    TRUE ~ "Others"                              # Catch-all for remaining cases
  ))

In [34]:
df |> count(category)

category,n
<chr>,<int>
MR only,44
Not Significant,7332
TWAS & MR,36
TWAS only,31


In [35]:
# dir.create('data')
saveRDS(list(df = df, unadj_cutoff = unadj_cutoff, axis_df= axis_df), 'data/TWAS_manhattan_plot_data.rds')